In [ ]:
from Archivos_Apoyo.AngleCouplingAnalyzer import AngleCouplingAnalyzer
ruta_entrenamiento_robot="logs_lift_leg/general_values_20251015_182343.pid660.csv"
ruta_recompensas="logs_lift_leg/rewards_20251015_182343.pid660.csv"
anglecouplinganalyzer=AngleCouplingAnalyzer()

In [ ]:
df_general=anglecouplinganalyzer.cargar_csv(ruta_entrenamiento_robot)
df_recompensas=anglecouplinganalyzer.cargar_csv(ruta_recompensas)

In [ ]:
angulos_joints=anglecouplinganalyzer.split_parameters_df(df_general, startswith="q_")
angular_speed=anglecouplinganalyzer.split_parameters_df(df_general, startswith="vel_")
torque=anglecouplinganalyzer.split_parameters_df(df_general, startswith="τ_reaction_")
forces=anglecouplinganalyzer.split_parameters_df(df_general, startswith="Forces_")
pressures=anglecouplinganalyzer.split_parameters_df(df_general, startswith="Pressure_")
coms=anglecouplinganalyzer.split_parameters_df(df_general, startswith="COM_")
zmps=anglecouplinganalyzer.split_parameters_df(df_general, startswith="ZMP_")
df_angle_speed = df_general.loc[:, df_general.columns.str.startswith(("q_", "vel_"))]
df_angle_com = df_general.loc[:, df_general.columns.str.startswith(("q_left_hip", "q_right_hip", "COM_"))]
df_presiones_velocidades=df_general.loc[:, df_general.columns.str.startswith(("vel_", "Pressure_"))]

In [ ]:
df_presiones_velocidades

In [ ]:
corr_angulos = anglecouplinganalyzer.correlacion_entre_params(angulos_joints)
corr_angular_speed = anglecouplinganalyzer.correlacion_entre_params(angular_speed)
corr_torque = anglecouplinganalyzer.correlacion_entre_params(torque)
corr_forces = anglecouplinganalyzer.correlacion_entre_params(forces)
corr_presiones=anglecouplinganalyzer.correlacion_entre_params(pressures)
corr_recompensas=anglecouplinganalyzer.correlacion_entre_params(df_recompensas.iloc[:,3:])
corr_coms=anglecouplinganalyzer.correlacion_entre_params(coms)
corr_zmp=anglecouplinganalyzer.correlacion_entre_params(zmps)

In [ ]:
corr_angulo_velocidad=anglecouplinganalyzer.correlacion_entre_params(df_angle_speed)
corr_velocidad_presiones=anglecouplinganalyzer.correlacion_entre_params(df_presiones_velocidades)
corr_com_angulo=anglecouplinganalyzer.correlacion_entre_params(df_angle_com)

In [ ]:
anglecouplinganalyzer.creacion_correlaciones(corr_angulos,"angulos")
anglecouplinganalyzer.creacion_correlaciones(corr_angular_speed, "v_angular")
anglecouplinganalyzer.creacion_correlaciones(corr_torque,"torque", annot=False)
anglecouplinganalyzer.creacion_correlaciones(corr_forces, "forces", annot=False)
anglecouplinganalyzer.creacion_correlaciones(corr_presiones, "presiones", annot=False)
anglecouplinganalyzer.creacion_correlaciones(corr_recompensas, "recompensas", annot=False)
anglecouplinganalyzer.creacion_correlaciones(corr_coms, "coms")
anglecouplinganalyzer.creacion_correlaciones(corr_zmp, "zmps")
anglecouplinganalyzer.creacion_correlaciones(corr_angulo_velocidad, "theta_vs_omega", annot=False)
anglecouplinganalyzer.creacion_correlaciones(corr_velocidad_presiones, "omega_vs_pressure", annot=False)
anglecouplinganalyzer.creacion_correlaciones(corr_com_angulo, "COM_vs_theta")

In [ ]:
# air_pose_viewer.py
import os
import pybullet as p
import pybullet_data
import time

ROBOT_NAME = "2_legged_human_like_robot20DOF"
# 1) Intenta cargar desde tu repo (Robots_Versiones). Si no, usa la copia absoluta.
CANDIDATE_PATHS = [
    f"Robots_Versiones/{ROBOT_NAME}.urdf",
    os.path.abspath("/mnt/data/2_legged_human_like_robot20DOF.urdf"),
]

def find_urdf():
    for path in CANDIDATE_PATHS:
        if os.path.exists(path):
            return path
    raise FileNotFoundError("No encuentro el URDF. Ajusta CANDIDATE_PATHS a tu estructura.")

if __name__ == "__main__":
    # GUI para ver el modelo
    cid = p.connect(p.GUI)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())

    # Sin gravedad → el robot “flota”
    p.resetSimulation()
    p.setGravity(0, 0, 0)
    p.setTimeStep(1.0 / 240.0)
    p.setRealTimeSimulation(0)

    # (Opcional) plano solo como referencia visual (no hay gravedad igualmente)
    plane = p.loadURDF("plane.urdf", [0, 0, 0])

    # Carga del bípedo un poco alto
    urdf_path = find_urdf()
    start_pos = [0, 0, 1.5]        # “en el aire”
    start_orn = p.getQuaternionFromEuler([0, 0, 0])
    robot = p.loadURDF(
        urdf_path,
        basePosition=start_pos,
        baseOrientation=start_orn,
        useFixedBase=True,         # base libre; al no haber gravedad no cae
        flags=p.URDF_USE_SELF_COLLISION
    )

    num_joints = p.getNumJoints(robot)

    # Desactiva motores por defecto en modo “fuerza 0” para que no intenten mantener una pose
    for j in range(num_joints):
        p.setJointMotorControl2(robot, j, p.VELOCITY_CONTROL, force=0)

    # Sliders de control por articulación (en radianes)
    sliders = []
    for j in range(num_joints):
        info = p.getJointInfo(robot, j)
        name = info[1].decode("utf-8")
        lo, hi = float(info[8]), float(info[9])
        # Si un joint no tiene límites sensatos (±inf), crea una ventana razonable
        if lo < -3.0 or lo == -1e10: lo = -1.5
        if hi >  3.0 or hi ==  1e10: hi =  1.5
        sliders.append((j, name, p.addUserDebugParameter(name, lo, hi, 0.0)))

    print("Controles listos: mueve los sliders para explorar la orientación de cada articulación.")
    print("Cierra la ventana para terminar.")

    # Bucle principal: aplica las posiciones desde los sliders y muestra ángulos
    while p.isConnected():
        # Aplica la posición articular de cada slider con resetJointState (pose “estática”)
        for j, name, sid in sliders:
            q = p.readUserDebugParameter(sid)
            p.resetJointState(robot, j, q)

        # (opcional) muestra por consola algunas juntas clave
        # states = p.getJointStates(robot, list(range(num_joints)))
        # angles = { p.getJointInfo(robot, j)[1].decode(): states[j][0] for j in range(num_joints) }
        # print(angles)

        p.stepSimulation()
        time.sleep(1.0/240.0)

    p.disconnect()

In [1]:
# world_axes_position_viewer.py
import os, time
import pybullet as p
import pybullet_data

ROBOT_NAME = "2_legged_human_like_robot20DOF"
CANDIDATE_PATHS = [
    f"Robots_Versiones/{ROBOT_NAME}.urdf",
    os.path.abspath("/mnt/data/2_legged_human_like_robot20DOF.urdf"),
]

def find_urdf():
    for path in CANDIDATE_PATHS:
        if os.path.exists(path):
            return path
    raise FileNotFoundError("Ajusta CANDIDATE_PATHS al path real del URDF.")

if __name__ == "__main__":
    cid = p.connect(p.GUI)
    p.setAdditionalSearchPath(pybullet_data.getDataPath())

    # Sim sin gravedad (robot “flota”)
    p.resetSimulation()
    p.setGravity(0, 0, 0)
    p.setTimeStep(1.0/240.0)
    p.setRealTimeSimulation(0)

    # Plano solo de referencia visual
    p.loadURDF("plane.urdf", [0, 0, 0])

    # Cargar robot un poco alto
    urdf_path = find_urdf()
    start_pos = [0.0, 0.0, 1.2]
    start_orn = p.getQuaternionFromEuler([0, 0, 0])
    robot = p.loadURDF(
        urdf_path,
        basePosition=start_pos,
        baseOrientation=start_orn,
        useFixedBase=False,
        flags=p.URDF_USE_SELF_COLLISION
    )

    # Apagar motores por defecto
    for j in range(p.getNumJoints(robot)):
        p.setJointMotorControl2(robot, j, p.VELOCITY_CONTROL, force=0)

    # ======== DIBUJOS DE REFERENCIA EN EL MUNDO ========
    # Ejes globales en el origen (rojo X, verde Y, azul Z)
    axis_len = 1.0
    p.addUserDebugLine([0,0,0], [axis_len,0,0], [1,0,0], lineWidth=3)  # X
    p.addUserDebugLine([0,0,0], [0,axis_len,0], [0,1,0], lineWidth=3)  # Y
    p.addUserDebugLine([0,0,0], [0,0,axis_len], [0,0,1], lineWidth=3)  # Z
    # Texto fijo “World Origin”
    p.addUserDebugText("World Origin (0,0,0)", [0.02, 0.02, 0.02], [1,1,1], textSize=1.3)

    # Línea desde el origen hasta la base (iremos reemplazándola)
    line_uid = p.addUserDebugLine([0,0,0], start_pos, [1,1,0], lineWidth=2)

    # HUD en pantalla con posición/orientación de la base
    # addUserDebugText con parentObjectUniqueId=-1 y lifeTime=0 no es HUD; para HUD usamos addUserDebugParameter?:
    # Truco: usamos addUserDebugText con lifeTime pequeño y lo vamos reescribiendo en cada frame.
    hud_uid = None

    # Sliders para recolocar BASE (opcional)
    bx_s = p.addUserDebugParameter("base_x (world)", -3.0, 3.0, start_pos[0])
    by_s = p.addUserDebugParameter("base_y (world)", -3.0, 3.0, start_pos[1])
    bz_s = p.addUserDebugParameter("base_z (world)",  0.0, 3.0, start_pos[2])
    lock_base = p.addUserDebugParameter("Lock base pose", 1, 0, 1)  # 0/1

    while p.isConnected():
        # Leer sliders y aplicar (si está bloqueado)
        bx = p.readUserDebugParameter(bx_s)
        by = p.readUserDebugParameter(by_s)
        bz = p.readUserDebugParameter(bz_s)
        if int(p.readUserDebugParameter(lock_base)) == 1:
            pos = [bx, by, bz]
            orn = p.getBasePositionAndOrientation(robot)[1]  # mantener orientación
            p.resetBasePositionAndOrientation(robot, pos, orn)

        # Forzar velocidades a cero para que no “salga disparado”
        p.resetBaseVelocity(robot, [0,0,0], [0,0,0])

        # Obtener pose actual de la base en el MUNDO
        pos, orn = p.getBasePositionAndOrientation(robot)
        roll, pitch, yaw = p.getEulerFromQuaternion(orn)

        # Actualizar línea desde origen → base
        p.removeUserDebugItem(line_uid)
        line_uid = p.addUserDebugLine([0,0,0], pos, [1,1,0], lineWidth=2)

        # Actualizar HUD con texto (x,y,z, yaw)
        if hud_uid is not None:
            p.removeUserDebugItem(hud_uid)
        hud_text = f"Base world pos (m): x={pos[0]:.3f}, y={pos[1]:.3f}, z={pos[2]:.3f} | yaw={yaw:.3f} rad"
        # Colocamos el texto cerca de la base para que sea fácil de ver
        text_pos = [pos[0], pos[1], pos[2] + 0.25]
        hud_uid = p.addUserDebugText(hud_text, text_pos, [1,1,1], textSize=1.6, lifeTime=0.2)
        print(f"x={pos[0]:.3f} m, y={pos[1]:.3f} m, z={pos[2]:.3f} m")
        p.stepSimulation()
        time.sleep(1.0/240.0)

x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 m, y=0.000 m, z=1.200 m
x=0.000 

error: Not connected to physics server.